In [1]:
# coding: utf-8
 
#print u'Выполняется обработка логов...'
 
# Импортируем необходимые библиотеки
import cx_Oracle
import numpy as np
import pandas as pd
import time
 
# Определяем время начала работы скрипта
# start_time = time.time()
 
 
 
# Создаем соединение с БД СУПП
dsn_tns = cx_Oracle.makedsn('10.0.8.117', '1521', service_name = 'emiasdb')
conn_SUPP = cx_Oracle.connect(user = 'CM_EMIAS', password = 'CM_EMIAS', dsn = dsn_tns)
# Создаем курсор - это специальный объект который делает запросы и получает их результаты
cursor_SUPP = conn_SUPP.cursor()
 
# Создаем соединение с БД ЕТД-2
conn_ETD2 = cx_Oracle.connect(user = 'ETD2', password = 'ETD2', dsn = dsn_tns, encoding = "UTF-8", nencoding = "UTF-8")
# Создаем курсор - это специальный объект который делает запросы и получает их результаты
cursor_ETD2 = conn_ETD2.cursor()
 
 
 
# Создаем датафрейм из выгрузки Graylog (поиск по 'addUserRoleRequest' OR 'removeUserRoleRequest')
frame = pd.read_csv('C:/graylog-search-result-relative.csv', header = 0)
 
 
 
# Удаляем лишние столбцы
frame.drop('timestamp', axis = 1, inplace = True)
frame.drop('source', axis = 1, inplace = True)
 
# Парсим дату запроса и добавляем ее в отдельный столбец
new = frame['message'].str.split(' ', n = 1, expand = True)
frame['Date']= new[0]
frame['message'] = new[1]
 
# Парсим время запроса и добавляем ее в отдельный столбец
new = frame['message'].str.split(' ', n = 1, expand = True)
frame['Time']= new[0]
frame['message'] = new[1]
 
# Парсим логин пользователя техподдержки и добавляем его в отдельный столбец
new = frame['message'].str.split('KPI_ADMIN/', n = 1, expand = True)
frame['Admin']= new[0]
frame['message'] = new[1]
new = frame['message'].str.split('<', n = 1, expand = True)
frame['Admin']= new[0]
frame['message'] = new[1]
 
# Добавляем отдельный столбец для типа операции добавлена/удалена роль
frame['Operation'] = ''
 
# Парсим тип операции и добавляем его в отдельный столбец
new = frame['message'].str.split('<soap:Body>', n = 1, expand = True)
frame['message'] = new[1]
new = frame['message'].str.split('Request', n = 1, expand = True)
frame['Operation']= new[0].str[8:]
frame['message'] = new[1]
 
# Парсим userId и добавляем его в отдельный столбец
new = frame['message'].str.split('<userId>', n = 1, expand = True)
frame['userId']= new[0]
frame['message'] = new[1]
new = frame['message'].str.split('<', n = 1, expand = True)
frame['userId']= new[0]
frame['message'] = new[1]
 
# Парсим userRoleCode и добавляем его в отдельный столбец
new = frame['message'].str.split('<userRoleCode>', n = 1, expand = True)
frame['userRoleCode']= new[0]
frame['message'] = new[1]
new = frame['message'].str.split('<', n = 1, expand = True)
frame['userRoleCode']= new[0]
frame['message'] = new[1]
 
# Парсим lpuId и добавляем его в отдельный столбец
new = frame['message'].str.split('<lpuId>', n = 1, expand = True)
frame['lpuId']= new[0]
frame['message'] = new[1]
new = frame['message'].str.split('<', n = 1, expand = True)
frame['lpuId']= new[0]
frame['message'] = new[1]
 
# Удаляем столбец с сообщениями
frame.drop('message', axis = 1, inplace = True)
 
# Сортируем данные по дате и времени в хронологическом порядке
frame.sort_values(['Date', 'Time'], ascending = [1, 1], inplace = True)
# Переиндексация строк
frame.reset_index(drop = True, inplace = True)
 
 
 
# Создаем фрейм для хранения уникальных userId
users = pd.DataFrame(frame['userId'].unique())
users.rename(index = str, columns = {0: 'userId'}, inplace = True)
users['LOGIN'] = ''

In [33]:
users

,userId,LOGIN
0,10162574,
1,13944823,
2,11150661,
3,10503390,
4,11961436,
...,...,...
637,11831242,
638,16788144,
639,15278320,
640,13372840,


In [36]:
# Функция определения логина пользователя по userID:
def getLogin(userId):
    # Делаем SELECT запрос к базе данных, используя обычный SQL-синтаксис
    cursor_SUPP.execute('SELECT LOGIN FROM emias_cluster.ACCOUNT WHERE LOGIN_ID= :userId', {'userId': str(userId)})
    # Получаем результат сделанного запроса
    result = cursor_SUPP.fetchone()
    if type(result) is tuple:
        return result[0]
    else:
        return ('---Пользователь не найден в БД---')

In [37]:
users['LOGIN']

0       
1       
2       
3       
4       
      ..
637     
638     
639     
640     
641     
Name: LOGIN, Length: 642, dtype: object

In [38]:
# Заполняем столбец LOGIN во фрейме usersa
for i in range(0, users.shape[0], 1):
    users['LOGIN'][i] = getLogin(users['userId'][i])

In [39]:

 

 

 
# Добавляем столбец LOGIN во фрейм с данными из csv
frame = pd.merge(frame, users, how = 'right', on = 'userId')
 
 
 
# Создаем фрейм для хранения уникальных userRoleCode
roles = pd.DataFrame(frame['userRoleCode'].unique())
roles.rename(index = str, columns = {0: 'userRoleCode'}, inplace = True)
roles['userRoleName'] = ''
 
# Функция определения наименования роли по userRoleCode:
def getUserRoleName(userRoleCode):
    # Делаем SELECT запрос к базе данных, используя обычный SQL-синтаксис
    cursor_ETD2.execute('SELECT USER_ROLE FROM USER_ROLE WHERE ID= :userRoleCode', {'userRoleCode': str(userRoleCode)})
    # Получаем результат сделанного запроса
    result = cursor_ETD2.fetchone()
    if type(result) is tuple:
        return result[0]
    else:
        return (u'---Роль не найдена в БД---')
 
 
 
 
# Заполняем столбец userRoleName во фрейме roles
for i in range(0, roles.shape[0], 1):
    roles['userRoleName'][i] = getUserRoleName(roles['userRoleCode'][i])
 
# Добавляем столбец userRoleName во фрейм с данными из csv
frame = pd.merge(frame, roles, how = 'right', on = 'userRoleCode')
 
 
 
# Перегруппировываем столбцы во фрейме
frame = frame[['Date', 'Time', 'Admin', 'Operation', 'userId', 'LOGIN', 'userRoleCode', 'userRoleName', 'lpuId']]
 
# Экспорт результатов в Excel
frame.to_excel(u'addedORremovedUserRole.xlsx')
 
# Выводим сообщение об успешном выполнении скрипта
print (u'Обработка логов завершена. Результаты см. в файле addedORremovedUserRole.xlsx')
 
 
 
# Закрываем соединение с БД
conn_SUPP.close()
conn_ETD2.close()
 
# Если надо вывести на экран статистику какой админ сколько ролей назначил и убрал
# print frame.groupby('Admin').size()
# Если необходимо посмотреть назначения / удаления ролей, совершенных конкретным пользователем
# print frame[frame['Admin'] == 'FBorzenkov']
 
 
# Выводим время работы скрипта
# print('--- %s seconds ---' % (time.time() - start_time))

Обработка логов завершена. Результаты см. в файле addedORremovedUserRole.xlsx
